In [1]:
#Import necessary Libraries
import numpy as np
import pickle
from osgeo import gdal
from osgeo import ogr
import os
import glob
import ntpath
import pandas as pd
import geopandas as gpd
from numpy import array
from shapely.geometry import shape
import re
import matplotlib.pyplot as plt
import networkx as nx
from shapely.geometry import LineString, MultiLineString
from pyproj import CRS

In [2]:
# Define the input npy file path
input_npy ="1.- Dataset_Input/Enschede3D/npy_EnschedeModel"

# Define the input geom_text file path
input_geomtxt = "1.- Dataset_Input/Enschede3D/geom_txt"

# Define the crs_epsg coordinate system RD_NEW: 28992, BGS2005 / CCS2005=7801
crs_epsg = 28992

#Dont forget to define the output path of the final shapefile.
output_shapefile = '2.- Final_Outputs/Enschede3D_Shapefiles/EnschedeModel'

In [3]:
# Define the math operation you want to apply to the coordinates
def modify_coords_backward(input_geomtxt, x, y, sz=255):
    # Open the file in read mode
    with open(input_geomtxt, 'r') as file:
        lines = file.readlines()

        # Extract the last line
        last_line = lines[-1]

        # Split the line by commas
        values = last_line.split(',')

        # Extract the values
        minx = float(values[0])
        maxx = float(values[1])
        miny = float(values[2])
        maxy = float(values[3])

        new_x = (((x * (maxx - minx) / sz) + minx))
        new_y = ((((sz - y) * (maxy - miny)/sz)) + miny)  # flip y coordinates to match annotation in HEAT paper

        return [new_x, new_y]
    
def extract_geometry(npy_path, geom_path):
    npy = np.load(npy_path, allow_pickle=True, encoding='bytes').tolist()
    corners = np.array(npy['corners'])
    edges = npy['edges']
    
    modified_corners = [modify_coords_backward(geom_path, pair[0], pair[1]) for pair in corners]
    
    G = nx.Graph()
    for i, modified_corner in enumerate(modified_corners):
        G.add_node(i, pos=(modified_corner[0], modified_corner[1]))
    for edge in edges:
        G.add_edge(edge[0], edge[1])
        
    lines = []
    # Iterate over each edge and retrieve the coordinates
    for edge in G.edges():
        node1 = modified_corners[edge[0]]
        node2 = modified_corners[edge[1]]
        line = LineString([node1, node2])
        lines.append(line)

    geometry = MultiLineString(lines=lines)
    return geometry

In [4]:
data = {
    "id": [],
    "name": [],
    "geometry": [],
}

npy_paths = glob.glob(f"{input_npy}/*.npy")
for idx, npy_path in enumerate(npy_paths):
    file_basename = ntpath.basename(npy_path).split('.')[0]
    geom_path = f"{input_geomtxt}/{file_basename}.txt"
        
    geometry = extract_geometry(npy_path, geom_path)
    data["id"].append(idx)
    data["name"].append(file_basename)
    data["geometry"].append(geometry)
    
gdf = gpd.GeoDataFrame(data, crs=CRS.from_epsg(crs_epsg))#take the values from the geom_text file
gdf.to_file(output_shapefile)